In [ ]:
import os
import pandas as pd
from collections import Counter

# Polku annotointikansioihin
base_path = "../data"

# Tyhjä DataFrame
df = pd.DataFrame()

# Luetaan tiedot
for folder in sorted(os.listdir(base_path)):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".txt"):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r') as f:
                    labels = [line.strip().split()[0] for line in f.readlines()]
                    label_str = ",".join(labels)
                    df.at[file, folder] = label_str if label_str else pd.NA

# Korvataan tyhjät merkkijonot NaN:iksi
df.replace("", pd.NA, inplace=True)

# Lista uudelleenkäsittelyä vaativista
uudelleen = []
tulokset = []

for filename, row in df.iterrows():
    values = row.dropna().tolist()
    if len(values) < 2:
        continue
    counts = Counter(values)
    if any(c > 1 for c in counts.values()) and len(counts) > 1:
        yleisimmat = [label for label, c in counts.items() if c > 1]
        uudelleen.append(filename)
        tulokset.append({
            "Tiedosto": filename,
            "Yleisimmät luokat": ", ".join(yleisimmat)
        })

# Tulosta määrä
print(f"Tiedostot, joissa oli eroja, mutta vähintään kaksi yhtenevää luokittelua: {len(uudelleen)} kpl\n")

# Tulosta yhteenvetotaulukko
tulos_df = pd.DataFrame(tulokset)
print(tulos_df)


In [ ]:
# Tunnista tiedostot, joissa kaikki luokitukset ovat keskenään erilaisia
ristiriitaiset = []

for filename, row in df.iterrows():
    values = row.dropna().tolist()
    if len(values) < 2:
        continue  # ei voi vertailla
    counts = Counter(values)
    if all(count == 1 for count in counts.values()):
        ristiriitaiset.append(filename)

# Tulosta ja tallenna
print(f"Pelkästään eroavia luokituksia: {len(ristiriitaiset)} tiedostossa\n")

for tiedosto in ristiriitaiset:
    print(tiedosto)


In [ ]:
# Tulostetaan eroavat tiedostot ja kaikki luokitukset

df_diff = df[df.nunique(axis=1, dropna=True) > 1]

for filename, row in df_diff.iterrows():
    print(f"\nTiedosto: {filename}")
    for folder in df.columns:
        luokka = row[folder]
        print(f"  {folder}: {luokka}")

print(f"Eroavia tiedostoja yhteensä: {len(df_diff)}")